In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random
import de_core_news_sm
import en_core_web_sm
# from torch.utils.tensorboard import SummaryWriter


In [15]:
spacy_eng = en_core_web_sm.load()
spacy_ger = de_core_news_sm.load()

In [16]:
'''Tokenizing Text'''

def tokenizer_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]
def tokenizer_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]

In [17]:
Field(tokenize=tokenize_ger, lower=True, init_token="<sos>", eos_token="<eos>")

NameError: name 'doc' is not defined

In [1]:
import torch